In [39]:
import cv2
import numpy as np
import mediapipe as mp

In [81]:
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    static_image_mode=True,
    refine_landmarks=True,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

In [41]:
def get_landmarks(image):
    landmarks = {}
    h,w = image.shape[:2]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_mesh_result = face_mesh.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if face_mesh_result.multi_face_landmarks:
        for i, landmark in enumerate(face_mesh_result.multi_face_landmarks[0].landmark): 
            x = landmark.x
            y = landmark.y
            relative_x = int(x * w)
            relative_y = int(y * h)
            landmarks[i+1] = (relative_x, relative_y)
    return landmarks


In [191]:
def get_coordinates(landmarks):
    coordinates = {
        "eye_left": [landmarks[30], [landmarks[158][0], landmarks[145][1]]],
        "eye_right": [landmarks[287], [landmarks[260][0], landmarks[381][1]]],
        # "shade": [landmarks[71], (landmarks[301][0], landmarks[301][1] + (landmarks[119][1] - landmarks[301][1]))]
        "shade": [(landmarks[140][0], landmarks[72][1]), landmarks[117], landmarks[346], landmarks[390]]
    }
    return coordinates

In [43]:
def add_effect(image, effect, icon_path, cordinates):
    item = cv2.imread(icon_path)
    pt1, pt2 = cordinates[effect]
    x, y, x_w, y_h = pt1[0], pt1[1], pt2[0], pt2[1]
    cropped = image[y:y_h, x:x_w, :]
    h, w, _ = cropped.shape
    item = cv2.resize(item, (w, h))
    blend = cv2.addWeighted(cropped, 0, item, 1.0, 0)
    
    return blend, x, y, x_w, y_h

In [44]:
def remove_whitespace(image, blend, x, y, threshold=225):
    for i in range(blend.shape[0]):
        for j in range(blend.shape[1]):
            for k in range(3):
                if blend[i][j][k] > threshold:
                    blend[i][j][k] = image[i + y][j + x][k]

In [45]:
def draw_iris_effects(image, coordinates):
    icon_path = "effects\\eye.png"
    for effect in ['eye_left', 'eye_right']:
        blend, x, y, x_w, y_h = add_effect(image, effect, icon_path, coordinates)
        remove_whitespace(
            image=image,
            blend=blend,
            x=x,
            y=y
        )
        image[y:y_h, x:x_w, :] = blend

In [46]:
# TODO: scale the effect and handle tilting of the effect with respect to head
def draw_specs_effect(image, coordinates):
    # icon_path = "effects\\specs.png"
    icon_path = "effects\\spec2.png"
    blend, x, y, x_w, y_h = add_effect(image, "shade", icon_path, coordinates)
    remove_whitespace(
        image=image,
        blend=blend,
        x=x,
        y=y
    )
    image[y:y_h, x:x_w, :] = blend
    

In [102]:
def draw_coordinates(image, points, color = [150, 0, 200]):
    neighbor_vector = [(0,0), (1,0), (0,1), (-1,0), (0, -1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    for pt in points:
        # pt = points[key]
        # for v in neighbor_vector:
        image[pt[1]][pt[0]] = color
            # image[pt[1]+v[0]][pt[0]+v[1]] = [i*50, 0, 0]
    return image

In [ ]:
def get_euclidean_distance(pt1, pt2):
    distance = Math.sqrt(
        pow((pt2[0] - pt1[0]), 2),
        pow((pt2[1] - pt1[1]), 2)
    )

In [192]:
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    
    if ret:
        height, width, _ = frame.shape
        image = cv2.resize(frame, (width//2, height//2))[:,::-1]
        landmarks = get_landmarks(image=image)
        coordinates = get_coordinates(landmarks=landmarks)
        # draw_iris_effects(image, coordinates)
        # draw_specs_effect(image, coordinates)
        # image = cv2.polylines(np.int32(image), np.int32(coordinates['shade']), True, (0, 225, 0), 1)
        image = draw_coordinates(image, coordinates['shade'], color=[255, 255, 255])
        
        cv2.imshow("Live face effects", image)
        if cv2.waitKey(5) & 0xFF == 27:
            cv2.destroyAllWindows() 
            cam.release()
            break

In [180]:
frame = cv2.imread("img.jpg")

cam = cv2.VideoCapture(0)
ret, frame = cam.read()


height, width, _ = frame.shape
# image = cv2.resize(frame, (width//2, height//2))[:,::-1]

In [181]:
landmarks = get_landmarks(image=frame)
coordinates = get_coordinates(landmarks=landmarks)

In [182]:
# for i in range(1, 469):
#     _img = draw_coordinates(image.copy(), list(landmarks.values()))
#     _img = draw_coordinates(_img, [landmarks[i]], [255, 255, 255])

#     cv2.imshow('Live face effects', image)
#     if cv2.waitKey(0) == 27:
#         cv2.destroyAllWindows()

In [188]:
image = draw_coordinates(frame, list(landmarks.values()))
image = draw_coordinates(
    image=image,
    points=[(landmarks[140][0], landmarks[72][1]), landmarks[117], landmarks[346], landmarks[390]],
    color=[255, 255, 255]
)

cv2.imshow('Live face effects', image)
if cv2.waitKey(0) == 27:
    cv2.destroyAllWindows()

In [187]:
cam.release()